
    Теперь решаем задачу регрессии - предскажем цены на недвижимость. 
    Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
    Построить случайный лес, вывести важность признаков
    Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
    В качестве решения: Jupyter notebook с кодом, комментариями и графиками



In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import auc, roc_curve, roc_auc_score


In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_Y = pd.read_csv('sample_submission.csv')


### 1 Информация о данных

In [6]:
print(train.shape,test.shape,test_Y.shape)

(1460, 81) (1459, 80) (1459, 2)


In [7]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [10]:
numeric=[f for f in train.columns if train.dtypes[f] != 'object']
num_features = train.select_dtypes(include=[np.number])

catagorical = [i for i in train.columns if train.dtypes[i] == 'object']
cat_features = train.select_dtypes(include=[object])
print(numeric)
print(catagorical)

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'Gara

#### В данных 80 признаков. 38 числовых. 19 признаков имеют пропущенные значения.

### Удаляем незначемые признаки и выделяем Target

In [11]:
train_Y = np.log1p(train.SalePrice.values)
#back = np.expm1(Y)

train_X = train.drop(['Id','SalePrice'], axis=1)
test_X = test.drop(['Id'], axis=1)

print(train_X.shape) 
print(test_X.shape)

ntrain = train_X.shape[0]
ntest = test_X.shape[0]
all_data = pd.concat((train_X, test_X)).reset_index(drop=True)

print("train : {}, train_X : {}, all_data : {} ".format(train.shape,train_X.shape,all_data.shape))

(1460, 79)
(1459, 79)
train : (1460, 81), train_X : (1460, 79), all_data : (2919, 79) 


### 2 Подготовка данных

### 2.1 Заполняем пропуски

In [12]:
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
# заполняем недостающие значения медианной площадью в этом районе
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)    
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)    
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data["Utilities"] = all_data["Utilities"].fillna("None")
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")


In [13]:
tp = all_data.isnull().sum()>0
tp[tp]

Series([], dtype: bool)

#### В данных пустот не осталось

### 2.2 Преобразуем категориальные признаки в числовые

In [14]:
# Некоторые признаки хоть и имеют числовой тип, но реально их природа категориальная
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
all_data['OverallCond'] = all_data['OverallCond'].astype(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

In [15]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

print('Shape all_data: {}'.format(all_data.shape))

Shape all_data: (2919, 79)


In [16]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2919, 223)


In [17]:
train_XX = all_data[:ntrain]
test_XX = all_data[ntrain:]
print('train_XX: {}, test_XX: {}'.format(train_XX.shape,test_XX.shape))

train_XX: (1460, 223), test_XX: (1459, 223)


### 3 Модель RandomForest

In [18]:
from sklearn.ensemble import RandomForestRegressor

In [19]:
model_1 = RandomForestRegressor(n_estimators=100, n_jobs=2)
model_1.fit(train_XX,train_Y)
pred_Y = np.expm1(model_1.predict(test_XX))
print(model_1.score(test_XX,np.log1p(test_Y['SalePrice'])))


-15.610765873208242


In [20]:
# Фичи отсортированные от важных к бесполезным
imp_f = np.sort(model_1.feature_importances_)
print(imp_f[::-1])

[5.54599126e-01 1.12261339e-01 4.41836279e-02 3.28871663e-02
 2.42768127e-02 2.22907918e-02 2.03285898e-02 1.61131041e-02
 1.58586075e-02 1.30278982e-02 1.04612039e-02 9.15967359e-03
 8.57804349e-03 6.28392677e-03 6.22197520e-03 5.76072788e-03
 5.38359470e-03 3.82709026e-03 3.71599542e-03 3.57062127e-03
 3.21721298e-03 2.95998643e-03 2.89305425e-03 2.82612757e-03
 2.78899270e-03 2.78819599e-03 2.67798023e-03 2.66808300e-03
 2.44380993e-03 2.33932316e-03 1.96680892e-03 1.82004285e-03
 1.72067259e-03 1.68706491e-03 1.56738369e-03 1.55045561e-03
 1.48247867e-03 1.27438941e-03 1.26440702e-03 1.25267028e-03
 1.24731345e-03 1.23115797e-03 1.22757669e-03 1.15463994e-03
 1.07906724e-03 1.07454389e-03 1.03407431e-03 1.03357866e-03
 9.99614986e-04 9.83937577e-04 9.12305947e-04 8.67903679e-04
 7.70535809e-04 7.50482871e-04 7.15429157e-04 7.07716594e-04
 6.49150226e-04 6.44763188e-04 6.11750130e-04 5.41473187e-04
 5.39158130e-04 4.90644563e-04 4.81091362e-04 4.80685890e-04
 4.80615192e-04 4.796434

### 4. Обучаем стекинг 3х моделей ( LinearRegression, KNeighborsRegressor, DecisionTreeRegressor )

In [21]:
from sklearn.ensemble import StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [22]:
estimators = [
                ('lr',   LinearRegression()),
                ('knr',  KNeighborsRegressor(n_neighbors=3)),
                ('tree', DecisionTreeRegressor())]

reg = StackingRegressor(estimators=estimators,
                        final_estimator=RandomForestRegressor(n_estimators=10))

reg.fit(train_XX,train_Y).score(test_XX,np.log1p(test_Y['SalePrice']))

-17.28571816187684